
# Hospital Patient Data Analysis (Pandas)

In [1]:
import pandas as pd
import numpy as np

patient_df = pd.read_csv("Patient_Data.csv")
billing_df = pd.read_csv("Billing_Data.csv")

print("Patient data shape:", patient_df.shape)
print("Billing data shape :", billing_df.shape)

patient_df.head()


Patient data shape: (6, 7)
Billing data shape : (5, 3)


,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


##summary using info()

In [2]:
patient_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


## Only billing columns and drop admin columns

In [3]:

admin_cols = ['ReceptionistID', 'CheckInTime']
patient_df_clean = patient_df.drop(columns=[c for c in admin_cols if c in patient_df.columns])

required_cols = ['PatientID', 'Department', 'Doctor', 'BillAmount']
patient_df_clean = patient_df_clean[required_cols].copy()

patient_df_clean.head()


,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


## Total bill amount per department

In [4]:
dept_total_bill = patient_df_clean.groupby("Department")["BillAmount"].sum()
dept_total_bill


,BillAmount
Department,
Cardiology,16200.0
Dermatology,0.0
Neurology,0.0
Orthopedics,7500.0


## Removing duplicate patient records based on PatientID

In [5]:
patient_df_nodup = patient_df_clean.drop_duplicates(subset="PatientID").copy()

print("Before removing duplicates:", patient_df_clean.shape)
print("After removing duplicates :", patient_df_nodup.shape)

patient_df_nodup.head()


Before removing duplicates: (6, 4)
After removing duplicates : (5, 4)


,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


## Filling missing BillAmount with mean

In [6]:
mean_bill = patient_df_nodup["BillAmount"].mean()
print("Mean BillAmount:", mean_bill)

patient_df_nodup["BillAmount"] = patient_df_nodup["BillAmount"].fillna(mean_bill)

patient_df_nodup.isnull().sum()


Mean BillAmount: 6233.333333333333


,0
PatientID,0
Department,0
Doctor,0
BillAmount,0


## Merge billing dataset with patient dataset on PatientID

In [7]:
merged_df = pd.merge(
    patient_df_nodup,
    billing_df,
    on="PatientID",
    how="left"   # keeps all patients even if billing info missing
)

print("Merged shape:", merged_df.shape)
merged_df.head()


Merged shape: (5, 6)


,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.000000,2000,3000
1,102,Neurology,Dr. John,6233.333333,1500,3500
2,103,Orthopedics,Dr. Lee,7500.000000,2500,5000
3,104,Cardiology,Dr. Smith,6200.000000,3000,3200
4,105,Dermatology,Dr. Rose,6233.333333,1000,4000


## Concatenate new patients for current week (row-wise)


If your assignment provides a “new patients” dataset, replace the sample below with your actual data.


In [8]:
new_patients = pd.DataFrame({
    "PatientID": [9001, 9002],
    "Department": ["Cardiology", "Neurology"],
    "Doctor": ["Dr. Rao", "Dr. Mehta"],
    "BillAmount": [12000, 8500]
})

combined_df = pd.concat([merged_df, new_patients], axis=0, ignore_index=True)

print("After adding new patients:", combined_df.shape)
combined_df.tail()


After adding new patients: (7, 6)


,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
2,103,Orthopedics,Dr. Lee,7500.000000,2500.0,5000.0
3,104,Cardiology,Dr. Smith,6200.000000,3000.0,3200.0
4,105,Dermatology,Dr. Rose,6233.333333,1000.0,4000.0
5,9001,Cardiology,Dr. Rao,12000.000000,NaN,NaN
6,9002,Neurology,Dr. Mehta,8500.000000,NaN,NaN


## new billing category columns (column-wise)

In [9]:
insurance_col = pd.Series(["Yes"] * len(combined_df), name="InsuranceCovered")

final_amount_col = pd.Series(
    np.where(insurance_col == "Yes", combined_df["BillAmount"] * 0.90, combined_df["BillAmount"]),
    name="FinalAmount"
)

final_df = pd.concat([
    combined_df.reset_index(drop=True),
    insurance_col.reset_index(drop=True),
    final_amount_col.reset_index(drop=True)
], axis=1)

final_df.head()


,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.000000,2000.0,3000.0,Yes,4500.0
1,102,Neurology,Dr. John,6233.333333,1500.0,3500.0,Yes,5610.0
2,103,Orthopedics,Dr. Lee,7500.000000,2500.0,5000.0,Yes,6750.0
3,104,Cardiology,Dr. Smith,6200.000000,3000.0,3200.0,Yes,5580.0
4,105,Dermatology,Dr. Rose,6233.333333,1000.0,4000.0,Yes,5610.0


## Final Output Checks

In [10]:
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PatientID         7 non-null      int64  
 1   Department        7 non-null      object 
 2   Doctor            7 non-null      object 
 3   BillAmount        7 non-null      float64
 4   InsuranceCovered  5 non-null      float64
 5   FinalAmount       5 non-null      float64
 6   InsuranceCovered  7 non-null      object 
 7   FinalAmount       7 non-null      float64
dtypes: float64(4), int64(1), object(3)
memory usage: 580.0+ bytes


In [11]:
final_df.isnull().sum()


,0
PatientID,0
Department,0
Doctor,0
BillAmount,0
InsuranceCovered,2
FinalAmount,2
InsuranceCovered,0
FinalAmount,0
